## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,73.22,70,100,23.42,light rain
1,1,Yako,BF,12.9667,-2.2667,73.60,85,46,6.58,scattered clouds
2,2,Hilo,US,19.7297,-155.0900,78.53,85,90,5.75,moderate rain
3,3,Butaritari,KI,3.0707,172.7902,82.49,71,8,8.97,light rain
4,4,Hobart,AU,-42.8794,147.3294,54.10,62,40,5.01,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,73.22,70,100,23.42,light rain
1,1,Yako,BF,12.9667,-2.2667,73.60,85,46,6.58,scattered clouds
2,2,Hilo,US,19.7297,-155.0900,78.53,85,90,5.75,moderate rain
7,7,Biltine,TD,14.5333,20.9167,78.40,70,99,8.61,overcast clouds
10,10,Rikitea,PF,-23.1203,-134.9692,73.40,81,46,11.14,light rain
11,11,Puerto Ayora,EC,-0.7393,-90.3518,71.17,89,33,5.48,scattered clouds
12,12,Carnarvon,AU,-24.8667,113.6333,66.27,58,40,14.97,scattered clouds
17,17,Nikolskoye,RU,59.7035,30.7861,69.40,89,69,6.42,broken clouds
18,18,Mahebourg,MU,-20.4081,57.7000,73.26,71,100,23.58,light rain
19,19,Pacific Grove,US,36.6177,-121.9166,69.03,88,40,1.01,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                252
City                   252
Country                252
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,73.22,70,100,23.42,light rain
1,1,Yako,BF,12.9667,-2.2667,73.60,85,46,6.58,scattered clouds
2,2,Hilo,US,19.7297,-155.0900,78.53,85,90,5.75,moderate rain
7,7,Biltine,TD,14.5333,20.9167,78.40,70,99,8.61,overcast clouds
10,10,Rikitea,PF,-23.1203,-134.9692,73.40,81,46,11.14,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,73.22,light rain,-20.3428,57.7575,
1,Yako,BF,73.60,scattered clouds,12.9667,-2.2667,
2,Hilo,US,78.53,moderate rain,19.7297,-155.0900,
7,Biltine,TD,78.40,overcast clouds,14.5333,20.9167,
10,Rikitea,PF,73.40,light rain,-23.1203,-134.9692,
11,Puerto Ayora,EC,71.17,scattered clouds,-0.7393,-90.3518,
12,Carnarvon,AU,66.27,scattered clouds,-24.8667,113.6333,
17,Nikolskoye,RU,69.40,broken clouds,59.7035,30.7861,
18,Mahebourg,MU,73.26,light rain,-20.4081,57.7000,
19,Pacific Grove,US,69.03,scattered clouds,36.6177,-121.9166,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,73.22,light rain,-20.3428,57.7575,Casa Tia Villa
1,Yako,BF,73.60,scattered clouds,12.9667,-2.2667,CADAAY
2,Hilo,US,78.53,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Biltine,TD,78.40,overcast clouds,14.5333,20.9167,Hôtel Trois Étoiles de Biltine
10,Rikitea,PF,73.40,light rain,-23.1203,-134.9692,People ThankYou
11,Puerto Ayora,EC,71.17,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Carnarvon,AU,66.27,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
17,Nikolskoye,RU,69.40,broken clouds,59.7035,30.7861,Tourist House - Sablino
18,Mahebourg,MU,73.26,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
19,Pacific Grove,US,69.03,scattered clouds,36.6177,-121.9166,Lovers Point Inn


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] !=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,73.22,light rain,-20.3428,57.7575,Casa Tia Villa
1,Yako,BF,73.60,scattered clouds,12.9667,-2.2667,CADAAY
2,Hilo,US,78.53,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Biltine,TD,78.40,overcast clouds,14.5333,20.9167,Hôtel Trois Étoiles de Biltine
10,Rikitea,PF,73.40,light rain,-23.1203,-134.9692,People ThankYou
11,Puerto Ayora,EC,71.17,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Carnarvon,AU,66.27,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
17,Nikolskoye,RU,69.40,broken clouds,59.7035,30.7861,Tourist House - Sablino
18,Mahebourg,MU,73.26,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
19,Pacific Grove,US,69.03,scattered clouds,36.6177,-121.9166,Lovers Point Inn


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather with Max Temp</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=clean_hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))